<a href="https://colab.research.google.com/github/caroline-cdz/Curso_EBAC_Analista_de_Dados/blob/main/Coleta_Dados_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --show-progress --continue -O ./credito.xlsx https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/credito.xlsx

--2023-12-05 23:22:45--  https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/credito.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748185 (731K) [application/octet-stream]
Saving to: ‘./credito.xlsx’

./credito.xlsx      100%[===================>] 730.65K  3.77MB/s    in 0.2s    

2023-12-05 23:22:46 (3.77 MB/s) - ‘./credito.xlsx’ saved [748185/748185]



In [ ]:
!pip install openpyxl

In [ ]:
from openpyxl import load_workbook

planilhas = load_workbook(filename='credito.xlsx')
planilha = planilhas.active

In [ ]:
# solução do exercício 1

from openpyxl import load_workbook
import csv

def extrair_e_salvar_dados(planilha, colunas_desejadas, nome_arquivo_csv):
    sheet = planilha.active
    cabecalhos = {sheet.cell(row=1, column=col_index).value: col_index for col_index in range(1, sheet.max_column + 1)}
    colunas_extraidas = {coluna: [] for coluna in colunas_desejadas}

    for row in sheet.iter_rows(min_row=2):
        for coluna_desejada in colunas_desejadas:
            col_index = cabecalhos[coluna_desejada]
            valor = row[col_index - 1].value
            colunas_extraidas[coluna_desejada].append(valor)

    with open(nome_arquivo_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        csv_writer.writerow(colunas_desejadas)

        for linha in zip(*colunas_extraidas.values()):
            csv_writer.writerow(linha)

planilhas = load_workbook(filename='credito.xlsx')
colunas_desejadas = ['id', 'default', 'sexo', 'idade', 'estado_civil']
nome_arquivo_csv = 'credito.csv'
extrair_e_salvar_dados(planilhas, colunas_desejadas, nome_arquivo_csv)

In [ ]:
import csv

def filtrar_e_salvar_csv(arquivo_entrada, arquivo_saida, colunas_desejadas, condicoes):

  with open(arquivo_entrada, mode='r', newline='', encoding='utf-8') as csv_in, \
       open(arquivo_saida, mode='w', newline='', encoding='utf-8') as csv_out:
        csv_reader = csv.reader(csv_in, delimiter=';')
        csv_writer = csv.writer(csv_out, delimiter=';')

        cabecalhos = next(csv_reader, None)

        if cabecalhos is None:
            return

        indices_colunas = [cabecalhos.index(coluna) for coluna in colunas_desejadas]

        csv_writer.writerow(colunas_desejadas)

        for linha in csv_reader:

          linha_dict = dict(zip(cabecalhos, linha))

          atende_condicoes = all(linha_dict[coluna] == condicao for coluna, condicao in condicoes.items())

          if atende_condicoes:
                csv_writer.writerow([linha_dict[coluna] for coluna in colunas_desejadas])

arquivo_entrada = 'credito.csv'
arquivo_saida = 'credito_filtrado.csv'
colunas_desejadas = ['id', 'sexo', 'idade', 'estado_civil', 'default']
condicoes = {'default': '1', 'estado_civil': 'solteiro'}
filtrar_e_salvar_csv(arquivo_entrada, arquivo_saida, colunas_desejadas, condicoes)


In [ ]:
# solução do exercício 2

import openpyxl
import json

planilha = openpyxl.load_workbook(filename='credito.xlsx')
planilha_ativa = planilha.active

escolaridades = [row[7].value for row in planilha_ativa.iter_rows(min_row=2)]
tipos_cartao = [row[11].value for row in planilha_ativa.iter_rows(min_row=2)]

escolaridades_unicas = list(set(escolaridades))
tipos_cartao_unicos = list(set(tipos_cartao))

credito = {
    'tipo_cartao': tipos_cartao_unicos,
    'escolaridade': escolaridades_unicas
}

credito_json = json.dumps(credito, indent=4)
print(credito_json)

{
    "tipo_cartao": [
        0,
        1,
        2,
        3,
        4,
        5,
        6
    ],
    "escolaridade": [
        "na",
        "$120K +",
        "$40K - $60K",
        "menos que $40K",
        "$80K - $120K",
        "$60K - $80K"
    ]
}


In [ ]:
%%writefile ebac.txt
MÍDIAS SOCIAIS
Instagram, Facebook, Youtube, LinkedIn

CURSOS
Software, Design, Marketing, Audiovisual, Programação & Data, Games

WEBINARS
Próximos, Anteriores

SOBRE
Sobre nós, Centro de carreiras, Vagas

CONTATO
WhatsApp +55 (11) 4200-2991
Telefone +55 (11) 3030-3200

BLOG
Design, Audiovisual, Marketing

Overwriting ebac.txt


In [ ]:
# solução do exercício 3 (bônus)

import csv
import re

def extrair_numeros_contato(linha):
  padrao_telefone = r'\+(\d{2,3})?(\d{8,12})'
  match = re.search(padrao_telefone, linha)
  if match:
    return match.group(0)
  else:
    return None

num_contato = []

with open(file='./ebac.txt', mode='r', encoding='utf8') as arq:
  linhas = arq.readlines()

linhas = list(filter(lambda l: l != '\n', map(lambda l: l.strip(), linhas)))

for linha in linhas:
  num = extrair_numeros_contato(linha)
  if num:
    num_contato.append(num)

dados_csv = [{'tipo': 'telefone', 'numero': num} for num in num_contato]